In [1]:
# -=-<[ Bismillahirrahmanirrahim ]>-=-
# -*- coding: utf-8 -*-
# @Date    : 2024-10-15 17:07:17
# @Author  : Dahir Muhammad Dahir (dahirmuhammad3@gmail.com)
# @Link    : link
# @Version : 1.0.0


from sqlalchemy import create_engine
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column, relationship, Session
from sqlalchemy import String, ForeignKey

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)



In [3]:

class Base(DeclarativeBase):
    pass

Base.metadata
Base.registry

In [4]:
import ulid

import inflect

from sqlalchemy import Date, DateTime, Integer
from sqlalchemy.ext.declarative import declared_attr
from sqlalchemy.sql.functions import func
from datetime import date as dt, datetime


get_plural = inflect.engine()


def get_new_ulid() -> str:
    return ulid.ulid()


class BaseMixin:
    @declared_attr  # type: ignore
    def __tablename__(cls) -> str:
        plural_name: str = get_plural.plural_noun(cls.__name__.lower())
        return plural_name

    id: Mapped[int] = mapped_column(
        Integer, primary_key=True, nullable=False, autoincrement=True
    )
    uuid: Mapped[str] = mapped_column(
        String(length=50), unique=True, nullable=False, default=get_new_ulid
    )
    date: Mapped[dt] = mapped_column(
        Date,
        index=True,
        default=dt.today,
        nullable=True,
        server_default=func.current_date(),
    )
    created_at: Mapped[datetime] = mapped_column(
        DateTime, index=True, server_default=func.now(), nullable=False
    )
    last_modified: Mapped[datetime] = mapped_column(
        DateTime, server_default=func.now(), onupdate=func.now(), nullable=False
    )


# class BaseModelMixin(BaseMixin):
#     @declared_attr
#     def created_by(cls: Any) -> Mapped[str]:
#         return mapped_column(String(50), ForeignKey("users.uuid"), nullable=False)

#     @declared_attr
#     def creator(cls: Any) -> Mapped[str]:
#         return relationship("User", lazy="joined")


class Yard(BaseMixin, Base):
    name: Mapped[str] = mapped_column(String(50), nullable=False)
    location: Mapped[str] = mapped_column(String(50), nullable=False)



class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[str | None]

    addresses: Mapped[list["Address"]] = relationship(back_populates="user")

    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"


class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))

    user: Mapped[User] = relationship("User", back_populates="addresses")

    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"



In [5]:
# create the tables in db

Base.metadata.create_all(engine)

2024-10-19 13:14:36,966 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-19 13:14:36,967 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("yards")
2024-10-19 13:14:36,967 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-19 13:14:36,968 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("yards")
2024-10-19 13:14:36,969 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-19 13:14:36,970 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-10-19 13:14:36,970 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-19 13:14:36,971 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-10-19 13:14:36,971 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-19 13:14:36,972 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-10-19 13:14:36,972 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-19 13:14:36,973 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-10-19 13:14:36,973 INFO sqlalchemy.engine.Engine [raw s

In [6]:
# inserting data using core
from sqlalchemy import insert

stmt = insert(User).values(name="dahir", fullname="dahir muhammad")
print(stmt)


INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [7]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()


2024-10-19 13:14:45,606 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-19 13:14:45,607 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2024-10-19 13:14:45,608 INFO sqlalchemy.engine.Engine [generated in 0.00206s] ('dahir', 'dahir muhammad')
2024-10-19 13:14:45,608 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
result.inserted_primary_key

(1,)

In [8]:
squidward = User(name="squidward", fullname="squidward tentacles")
krabs = User(name="krabs", fullname="eugene krabs")
spongebob = User(name="spongebob", fullname="spongebob squarepants")

In [9]:
with Session(engine) as session:
    session.add(squidward)
    session.add(krabs)
    session.add(spongebob)
    session.commit()



2024-10-18 20:56:41,887 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-18 20:56:41,889 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-10-18 20:56:41,889 INFO sqlalchemy.engine.Engine [generated in 0.00011s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('squidward', 'squidward tentacles')
2024-10-18 20:56:41,890 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-10-18 20:56:41,890 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('krabs', 'eugene krabs')
2024-10-18 20:56:41,890 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-10-18 20:56:41,890 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('spongebob', 'spongebob squarepants')
2024-10-18 20:56:41,891 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
from sqlalchemy import select
stmt = select(User.fullname).where(User.id == 2)

with Session(engine) as session:
    result = session.execute(stmt).scalar_one()

print(result)

2024-10-18 20:56:41,896 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-18 20:56:41,898 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2024-10-18 20:56:41,899 INFO sqlalchemy.engine.Engine [generated in 0.00078s] (2,)
2024-10-18 20:56:41,900 INFO sqlalchemy.engine.Engine ROLLBACK
squidward tentacles


In [11]:
stmt = select(User).where(User.name == "spongebob")
result = session.execute(stmt)

2024-10-18 20:56:41,905 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-18 20:56:41,907 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-10-18 20:56:41,908 INFO sqlalchemy.engine.Engine [generated in 0.00064s] ('spongebob',)


In [12]:
print(result.all())

[(User(id=4, name='spongebob', fullname='spongebob squarepants'),)]


In [8]:

my_yard = Yard(name="my yard", location="my location")

with Session(engine) as session:
    session.add(my_yard)
    session.commit()



2024-10-19 13:32:44,013 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-19 13:32:44,016 INFO sqlalchemy.engine.Engine INSERT INTO yards (name, location, uuid, date) VALUES (?, ?, ?, ?) RETURNING id, created_at, last_modified
2024-10-19 13:32:44,017 INFO sqlalchemy.engine.Engine [generated in 0.00139s] ('my yard', 'my location', '01JAJC82FF35VHMMG45YHQ77H3', '2024-10-19')
2024-10-19 13:32:44,018 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:

from google.cloud import storage
from google.cloud.client import Client

client = Client()

cli = storage.
